<a href="https://colab.research.google.com/github/prateekjain24/Public/blob/main/notebooks/%5BLatest%5DFE_vn_to_en_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tiktoken --upgrade
! pip install cohere --upgrade
! pip install openai --upgrade
#! pip install git+https://github.com/openai/whisper.git
! pip install streamlit
! pip install transformers soundfile
! pip install pyngrok
! pip install accelerate
! pip install deepgram-sdk
! pip install python-magic
! pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']= userdata.get('openai_key')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Colab Notebooks/pythontest IAM & Admin.json'
os.environ['DEEPGRAM_API_KEY'] = userdata.get('deepkram_key')
os.environ['HF_TOKEN'] = userdata.get('HF_key')
os.environ['GROQ_API_KEY'] = userdata.get('Groq')

In [ ]:
!wget -q -O - ipv4.icanhazip.com

104.197.28.146


In [ ]:
%%writefile app.py
# Import dependencies
import os
import magic
from google.cloud import translate_v2 as translate
import openai
from openai import OpenAI
# Set API keys
openai.api_key = os.environ['OPENAI_API_KEY']
deepgram_key = os.environ['DEEPGRAM_API_KEY']
hf_token = os.environ['HF_TOKEN']
from deepgram import (
    DeepgramClient,
    DeepgramClientOptions,
    PrerecordedOptions,
    FileSource,
)
import asyncio, json, os
import streamlit as st
import uuid
import soundfile as sf
from transformers import WhisperForConditionalGeneration, WhisperTokenizer
import requests
deepgram = DeepgramClient()
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from datasets import load_dataset

# Download Whisper 3 large from Hugging Face
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# Function for Transcription via Deepgram
def transcribe_with_deepgram(audio_path):
    # Call the transcribe_file method on the prerecorded class
    with open(audio_path, "rb") as file:
      buffer_data = file.read()

    payload: FileSource = {
        "buffer": buffer_data,
    }

    options = PrerecordedOptions(
       smart_format=True,
       model= 'whisper-large',
       language= 'vi',  # ISO 639-1 code for Vietnamese
    )
    response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options, timeout=300)
    response1 = response.to_json()
    print(type(response1))
    response2 = json.loads(response1)

    st.write("Transcribed Text:")
    st.write(response2['results']['channels'][0]['alternatives'][0]['transcript'])
    return response2['results']['channels'][0]['alternatives'][0]['transcript']

# Function for Transcription via OpenAI Whisper2 APIs
def transcribe_with_openai_whisper(audio_path):
    # Open the audio file
    audio_file = open(audio_path, "rb")
    client = OpenAI(api_key=openai.api_key)
    transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    language="vi",
    response_format="text"
  )
    st.write("Transcribed Text:")
    st.write(transcript)
    return transcript

# Function for Transcription via Local downloaded Whisper3-Large
# Needs a GPU machine with large memory min T4 ideally V100
def transcribe_with_local(audio_file_path):
    result = pipe(audio_file_path, generate_kwargs={"language": "Vietnamese"})
    st.write("Transcribed Text:")
    st.write(result["text"])
    return result["text"]

# Function for Translate to English
def translate_to_english(text):
    # Set the path to the service account key file

    # Initialize the Google Cloud Translate client
    translate_client = translate.Client()

    # Translate the text
    result = translate_client.translate(text, target_language='en')
    st.write("Translated Text:")
    st.write(result['translatedText'])
    return result['translatedText']

# Function for summarisation
def summarize_text(text1):
    # Summarize the text using GPT-4 turbo model
    client = OpenAI(api_key=openai.api_key)

    # Define the input prompts
    agent_role = "You are a sales trainer specializing in training sales team for real estate marketplace, your role encompasses evaluating telesales sales call transcripts. Your evaluations are instrumental in enhancing sales strategies for Batdongsan.com.vn, a leading real estate platform in Vietnam, akin to Zillow. This platform enables customers to invest in various listing products, such as standard listings, VIP listings (VIP Silver, VIP Gold & VIP Diamond), Boost, and Sponsored Listings, through telesales. Telesales generally ask property agents to add money to there account so that they can use these products. The company occasionally launches promotional campaigns, offering additional money or discounts on product usage."

    agent_responsibility =("Perfect. Also these are your responsibilities\n Scoring Transcripts: Evaluate each sales call transcript on a 10-point scale, focusing on critical aspects that influence sales effectiveness. Your evaluation criteria include:\n"
    "1. Clarity & Detail\n"
    "2. Structure & Coherence\n"
    "3. Persuasiveness\n"
    "4. Customer Understanding & Empathy\n"
    "5. Professionalism\n"
    "6. Closing Ability\n"
    "Providing Constructive Feedback: Alongside the numerical score, offer concise, structured feedback in English. Highlight areas of strength and suggest improvements to enhance sales performance. Keep it short and to the point"
    )

    agent_considerations = ("Sure. I will keep in mind these considerations:\n Language Proficiency: You will encounter transcripts in both Vietnamese and English. For Vietnamese transcripts, ensure a reliable translation to English before evaluation.\n"
    "Contextual Understanding: Familiarize yourself with the specific terminologies and practices within the Vietnamese real estate market. For instance, `newsletter` refers to a listing on Batdongsan.com.vn, and Zalo is the preferred messaging app for communication. Chotot & Nhatot are the major competitors\n"
    "Structured Feedback: Structure your feedback in bullet points, ensuring it is brief. Pay particular attention to identifying the speaker (salesperson or customer) and the context of their statements."
    )

    agent_objective = "Objective: Your critical assessments aim to refine the sales teams approach, enhancing their ability to engage effectively with clients and close sales successfully. Your feedback will serve as a valuable tool for ongoing training and development within the sales department.KEEP FEEDBACK SHORT"

    completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    seed=1221,
    messages=[
          {"role": "system", "content": agent_role},
          {"role": "user", "content": agent_objective},
          {"role": "assistant", "content": agent_considerations},
          {"role": "user", "content": agent_responsibility},
          {"role": "assistant", "content": "Sure, please provide the transcript."},
          {"role": "user", "content": text1}
           ]
         )
    return completion.choices[0].message.content


async def speech_to_text(audio_file_path, processing_model):

    # Function 1: Convert speech to text
    if processing_model == 'OpenAI':
      transcribed_text = transcribe_with_openai_whisper(audio_file_path)
    else:
      transcribed_text = transcribe_with_local(audio_file_path)

    # Function 2: Translate to English
    translated_text = translate_to_english(transcribed_text)

    # Function 3: Summarize using openAI
    summary = summarize_text(translated_text)

    # Return the processed text
    return summary

def speech_text(audio_file_path, processing_option):
    return asyncio.run(speech_to_text(audio_file_path, processing_option))

# Set the title and header
st.title("Sale - Call to summary Demo App")
st.header("Upload an Vietnamese audio file to process:")

# Select the transcription model
processing_option = st.radio(
    "Choose a transcription option:",
    ('Local', 'OpenAI')
)
# Upload audio file
uploaded_file = st.file_uploader("Choose Vietnamese audio file")


# Process uploaded file
# Process uploaded file
if uploaded_file is not None:
    # Read the content of the uploaded file
    file_content = uploaded_file.read()

    # Use magic to detect the file type
    file_type = magic.from_buffer(file_content, mime=True)

    # Assign the correct extension based on the detected file type
    if file_type == "audio/mpeg":
        extension = ".mp3"
    elif file_type == "audio/x-wav":
        extension = ".wav"
    else:
        raise ValueError("Unsupported file type")

    # Save the audio file with the correct extension
    filename = str(uuid.uuid4()) + extension
    with open(filename, "wb") as f:
        f.write(file_content)

    # Process the audio file
    processed_text = speech_text(filename, processing_option)

    # Display the processed text
    st.write("Processed Text:")
    st.write(processed_text)
####################################################

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.197.28.146:8501

npx: installed 22 in 4.408s
your url is: https://bitter-numbers-think.loca.lt
2024-03-26 07:25:23.130747: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 07:25:23.130807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 07:25:23.132234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 07:25:24.797471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have b

# Some Test code below - Ignore

In [ ]:
agent_role = "As a sales trainer specializing in the real estate marketplace, your role encompasses evaluating telesales sales call transcripts. Your evaluations are instrumental in enhancing sales strategies for Batdongsan.com.vn, a leading real estate platform in Vietnam, akin to Zillow. This platform enables customers to invest in various listing products, such as standard listings, VIP listings, Boost, and Sponsored Listings, through telesales. The company occasionally launches promotional campaigns, offering additional credits or discounts on product usage."

agent_responsibility =("Responsibility\n Scoring Transcripts: Evaluate each sales call transcript on a 10-point scale, focusing on critical aspects that influence sales effectiveness. Your evaluation criteria include:\n"
"1. Clarity & Detail\n"
"2. Structure & Coherence\n"
"3. Persuasiveness\n"
"4. Customer Understanding & Empathy\n"
"5. Professionalism\n"
"6. Closing Ability\n"
"Providing Constructive Feedback: Alongside the numerical score, offer concise, structured feedback in English. Highlight areas of strength and suggest improvements to enhance sales performance."
)

agent_considerations = ("COnsideration\n Language Proficiency: You will encounter transcripts in both Vietnamese and English. For Vietnamese transcripts, ensure a reliable translation to English before evaluation.\n"
"Contextual Understanding: Familiarize yourself with the specific terminologies and practices within the Vietnamese real estate market. For instance, `newsletter` refers to a listing on Batdongsan.com.vn, and Zalo is the preferred messaging app for communication. Chotot & Nhatot are the major competitors\n"
"Structured Feedback: Structure your feedback in bullet points, ensuring it is brief yet comprehensive. Pay particular attention to identifying the speaker (salesperson or customer) and the context of their statements."
)

agent_objective = "Objective: Your critical assessments aim to refine the sales teams approach, enhancing their ability to engage effectively with clients and close sales successfully. Your feedback will serve as a valuable tool for ongoing training and development within the sales department.KEEP FEEDBACK SHORT"

In [ ]:
from pathlib import Path
def voiceover_with_opneai():
    # Set OpenAI API key & intialize OpenAI Client
    client = OpenAI(api_key=openai.api_key)
    # Set path where audio file will be saved
    speech_file_path = "/content/drive/MyDrive/Colab Notebooks/speech4.mp3"
    response = client.audio.speech.create(
    model="tts-1", #tts-hd
    voice="alloy", #
    input= summary
    )
    audio_content = response.content
    with open(speech_file_path,"wb") as f:
      f.write(audio_content)

voiceover_with_opneai()

In [ ]:
# Import dependencies
import os
import magic
from google.cloud import translate_v2 as translate
import openai
from openai import OpenAI
# Set API keys
openai.api_key = os.environ['OPENAI_API_KEY']
deepgram_key = os.environ['DEEPGRAM_API_KEY']
hf_token = os.environ['HF_TOKEN']
from deepgram import (
    DeepgramClient,
    DeepgramClientOptions,
    PrerecordedOptions,
    FileSource,
)
import asyncio, json, os
import streamlit as st
import uuid
import soundfile as sf
from transformers import WhisperForConditionalGeneration, WhisperTokenizer
import requests
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from datasets import load_dataset

# Download Whisper 3 large from Hugging Face
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"
model_cache = "/content/drive/MyDrive/Colab Notebooks/Whisper3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, cache_dir=model_cache
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# Function for Transcription via Local downloaded Whisper3-Large
# Needs a GPU machine with large memory min T4 ideally V100
def transcribe_with_local(audio_file_path):
    result = pipe(audio_file_path, generate_kwargs={"language": "Vietnamese"})
    return result["text"]

# Function for Translate to English
def translate_to_english(text):
    # Initialize the Google Cloud Translate client
    translate_client = translate.Client()

    # Translate the text
    result = translate_client.translate(text, target_language='en')
    return result['translatedText']

# Function for summarisation
def summarize_text(text1):
    # Summarize the text using GPT-4 turbo model
    client = OpenAI(api_key=openai.api_key)

    # Define the input prompts
    agent_role = "You are a sales trainer specializing in training sales team for real estate marketplace, your role encompasses evaluating telesales sales call transcripts. Your evaluations are instrumental in enhancing sales strategies for Batdongsan.com.vn, a leading real estate platform in Vietnam, akin to Zillow. This platform enables customers to invest in various listing products, such as standard listings, VIP listings (VIP Silver, VIP Gold & VIP Diamond), Boost, and Sponsored Listings, through telesales. Telesales generally ask property agents to add money to there account so that they can use these products. The company occasionally launches promotional campaigns, offering additional money or discounts on product usage."

    agent_responsibility =("Perfect. Also these are your responsibilities\n Scoring Transcripts: Evaluate each sales call transcript on a 10-point scale, focusing on critical aspects that influence sales effectiveness. Your evaluation criteria include:\n"
    "1. Clarity & Detail\n"
    "2. Structure & Coherence\n"
    "3. Persuasiveness\n"
    "4. Customer Understanding & Empathy\n"
    "5. Professionalism\n"
    "6. Closing Ability\n"
    "Providing Constructive Feedback: Alongside the numerical score, offer concise, structured feedback in English. Highlight areas of strength and suggest improvements to enhance sales performance. Keep it short and to the point"
    )

    agent_considerations = ("Sure. I will keep in mind these considerations:\n Language Proficiency: You will encounter transcripts in both Vietnamese and English. For Vietnamese transcripts, ensure a reliable translation to English before evaluation.\n"
    "Contextual Understanding: Familiarize yourself with the specific terminologies and practices within the Vietnamese real estate market. For instance, `newsletter` refers to a listing on Batdongsan.com.vn, and Zalo is the preferred messaging app for communication. Chotot & Nhatot are the major competitors\n"
    "Structured Feedback: Structure your feedback in bullet points, ensuring it is brief. Pay particular attention to identifying the speaker (salesperson or customer) and the context of their statements."
    )

    agent_objective = "Objective: Your critical assessments aim to refine the sales teams approach, enhancing their ability to engage effectively with clients and close sales successfully. Your feedback will serve as a valuable tool for ongoing training and development within the sales department.KEEP FEEDBACK SHORT"

    completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    seed=1221,
    messages=[
          {"role": "system", "content": agent_role},
          {"role": "user", "content": agent_objective},
          {"role": "assistant", "content": agent_considerations},
          {"role": "user", "content": agent_responsibility},
          {"role": "assistant", "content": "Sure, please provide the transcript."},
          {"role": "user", "content": text1}
           ]
         )
    return completion.choices[0].message.content


async def speech_to_text(audio_file_path):

    transcribed_text = transcribe_with_local(audio_file_path)
    # Function 3: Summarize using openAI
    summary = summarize_text(transcribed_text)
    # Return the processed text
    return summary



preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Specify the directory containing the audio files
audio_files_directory = "/content/drive/MyDrive/Colab Notebooks/Record_Jan/Call Group 16"
text_files_directory = "/content/drive/MyDrive/Colab Notebooks/Record_Jan/Summary Group 16"
# List all files in the specified directory
for file_name in os.listdir(audio_files_directory):
    # Construct the full file path
    file_path = os.path.join(audio_files_directory, file_name)

    # Check if the current item is a file (and not a directory)
    if os.path.isfile(file_path) and file_name.lower().endswith(('.mp3', '.wav')):
        # Call the process_audio function on the file
        summary1 = await speech_to_text(file_path)

        # Construct the new file name with .txt extension
        base_name = os.path.splitext(file_name)[0]
        text_file_name = base_name + ".txt"
        text_file_path = os.path.join(text_files_directory, text_file_name)

        # Save the transcription to a text file
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(summary1)

        print(f"Transcription saved to {text_file_path}")

print("Summary completed.")

In [ ]:
# Specify the directory containing the audio files
audio_files_directory = "/content/drive/MyDrive/Colab Notebooks/Record_Jan/Call Group 17"
text_files_directory = "/content/drive/MyDrive/Colab Notebooks/Record_Jan/Summary Group 17"
# List all files in the specified directory
for file_name in os.listdir(audio_files_directory):
    # Construct the full file path
    file_path = os.path.join(audio_files_directory, file_name)

    # Check if the current item is a file (and not a directory)
    if os.path.isfile(file_path) and file_name.lower().endswith(('.mp3', '.wav')):
        # Call the process_audio function on the file
        summary1 = await speech_to_text(file_path)

        # Construct the new file name with .txt extension
        base_name = os.path.splitext(file_name)[0]
        text_file_name = base_name + ".txt"
        text_file_path = os.path.join(text_files_directory, text_file_name)

        # Save the transcription to a text file
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(summary1)

        print(f"Transcription saved to {text_file_path}")

print("Summary completed.")

In [ ]:
%%writefile app.py
####################################################
# Import dependencies
import os
import magic
from google.cloud import translate_v2 as translate
import openai
from openai import OpenAI
# Set API keys
openai.api_key = os.environ['OPENAI_API_KEY']
deepgram_key = os.environ['DEEPGRAM_API_KEY']
hf_token = os.environ['HF_TOKEN']
from deepgram import (
    DeepgramClient,
    DeepgramClientOptions,
    PrerecordedOptions,
    FileSource,
)
from groq import Groq
import asyncio, json, os
import streamlit as st
import uuid
import soundfile as sf
from transformers import WhisperForConditionalGeneration, WhisperTokenizer
import requests
deepgram = DeepgramClient()
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from datasets import load_dataset

# Download Whisper 3 large from Hugging Face
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"
model_cache = "/content/drive/MyDrive/Colab Notebooks/Whisper3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, cache_dir=model_cache
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# Function for Transcription via Deepgram
def transcribe_with_deepgram(audio_path):
    # Call the transcribe_file method on the prerecorded class
    with open(audio_path, "rb") as file:
      buffer_data = file.read()

    payload: FileSource = {
        "buffer": buffer_data,
    }

    options = PrerecordedOptions(
       smart_format=True,
       model= 'whisper-large',
       language= 'vi',  # ISO 639-1 code for Vietnamese
    )
    response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options, timeout=300)
    response1 = response.to_json()
    print(type(response1))
    response2 = json.loads(response1)

    st.write("Transcribed Text:")
    st.write(response2['results']['channels'][0]['alternatives'][0]['transcript'])
    return response2['results']['channels'][0]['alternatives'][0]['transcript']

# Function for Transcription via OpenAI Whisper2 APIs
def transcribe_with_openai_whisper(audio_path):
    # Open the audio file
    audio_file = open(audio_path, "rb")
    client = OpenAI(api_key=openai.api_key)
    transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    language="vi",
    response_format="text"
  )
    st.write("Transcribed Text:")
    st.write(transcript)
    return transcript

# Function for Transcription via Local downloaded Whisper3-Large
# Needs a GPU machine with large memory min T4 ideally V100
def transcribe_with_local(audio_file_path):
    result = pipe(audio_file_path, generate_kwargs={"language": "Vietnamese"})
    st.write("Transcribed Text:")
    st.write(result["text"])
    return result["text"]

# Function for Translate to English
def translate_to_english(text):
    # Set the path to the service account key file

    # Initialize the Google Cloud Translate client
    translate_client = translate.Client()

    # Translate the text
    result = translate_client.translate(text, target_language='en')
    st.write("Translated Text:")
    st.write(result['translatedText'])
    return result['translatedText']

# Function for summarisation OpenAI
def summarize_text(text1):
    # Summarize the text using GPT-4 turbo model
    client = OpenAI(api_key=openai.api_key)

    # Define the input prompts
    agent_role = "You are a sales trainer specializing in training sales team for real estate marketplace, your role encompasses evaluating telesales sales call transcripts. Your evaluations are instrumental in enhancing sales strategies for Batdongsan.com.vn, a leading real estate platform in Vietnam, akin to Zillow. This platform enables customers to invest in various listing products, such as standard listings, VIP listings (VIP Silver, VIP Gold & VIP Diamond), Boost, and Sponsored Listings, through telesales. Telesales generally ask property agents to add money to there account so that they can use these products. The company occasionally launches promotional campaigns, offering additional money or discounts on product usage."

    agent_responsibility =("Perfect. Also these are your responsibilities\n Scoring Transcripts: Evaluate each sales call transcript on a 10-point scale for each of the following criteria, focusing on critical aspects that influence sales effectiveness. Your evaluation criteria include:\n"
    "1. Clarity & Detail\n"
    "2. Structure & Coherence\n"
    "3. Persuasiveness\n"
    "4. Customer Understanding & Empathy\n"
    "5. Professionalism\n"
    "6. Closing Ability\n"
    "Providing Constructive Feedback: Alongside the numerical score, offer concise, structured feedback in English. Highlight areas of strength and suggest improvements to enhance sales performance. Keep it short and to the point"
    )

    agent_considerations = ("Sure. I will keep in mind these considerations:\n Language Proficiency: You will encounter transcripts in both Vietnamese and English. For Vietnamese transcripts, ensure a reliable translation to English before evaluation.\n"
    "Contextual Understanding: Familiarize yourself with the specific terminologies and practices within the Vietnamese real estate market. For instance, `newsletter` refers to a listing on Batdongsan.com.vn, and Zalo is the preferred messaging app for communication. Chotot & Nhatot are the major competitors\n"
    "Structured Feedback: Structure your feedback in bullet points, ensuring it is brief. Pay particular attention to identifying the speaker (salesperson or customer) and the context of their statements."
    )

    agent_objective = "Objective: Your critical assessments aim to refine the sales teams approach, enhancing their ability to engage effectively with clients and close sales successfully. Your feedback will serve as a valuable tool for ongoing training and development within the sales department.KEEP FEEDBACK SHORT"

    completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    seed=1221,
    messages=[
          {"role": "system", "content": agent_role},
          {"role": "user", "content": agent_objective},
          {"role": "assistant", "content": agent_considerations},
          {"role": "user", "content": agent_responsibility},
          {"role": "assistant", "content": "Sure, please provide the transcript."},
          {"role": "user", "content": text1}
           ]
         )
    return completion.choices[0].message.content

# Function for summarisation Groq
def summarize_text_groq(text1):
    # Summarize the text using Groq / Mixtral 8X7b
    client = Groq(
    api_key=os.environ['GROQ_API_KEY'],
    )

    # Define the input prompts
    agent_role = "You are a sales trainer specializing in training sales team for real estate marketplace, your role encompasses evaluating telesales sales call transcripts. Your evaluations are instrumental in enhancing sales strategies for Batdongsan.com.vn, a leading real estate platform in Vietnam, akin to Zillow. This platform enables customers to invest in various listing products, such as standard listings, VIP listings (VIP Silver, VIP Gold & VIP Diamond), Boost, and Sponsored Listings, through telesales. Telesales generally ask property agents to add money to there account so that they can use these products. The company occasionally launches promotional campaigns, offering additional money or discounts on product usage."

    agent_responsibility =("Perfect. Also these are your responsibilities\n Scoring Transcripts: Evaluate each sales call transcript on a 10-point scale, focusing on critical aspects that influence sales effectiveness. Your evaluation criteria include:\n"
    "1. Clarity & Detail(Score out of 10)\n"
    "2. Structure & Coherence(Score out of 10)\n"
    "3. Persuasiveness(Score out of 10)\n"
    "4. Customer Understanding & Empathy(Score out of 10)\n"
    "5. Professionalism(Score out of 10)\n"
    "6. Closing Ability(Score out of 10)\n"
    "Providing Constructive Feedback: Alongside the numerical score, offer concise, structured feedback in English. Highlight areas of strength and suggest improvements to enhance sales performance. Keep it short and to the point"
    )

    agent_considerations = ("Sure. I will keep in mind these considerations:\n Language Proficiency: You will encounter transcripts in both Vietnamese and English. For Vietnamese transcripts, ensure a reliable translation to English before evaluation.\n"
    "Contextual Understanding: Familiarize yourself with the specific terminologies and practices within the Vietnamese real estate market. For instance, `newsletter` refers to a listing on Batdongsan.com.vn, and Zalo is the preferred messaging app for communication. Chotot & Nhatot are the major competitors\n"
    "Structured Feedback: Structure your feedback in bullet points, ensuring it is brief. Pay particular attention to identifying the speaker (salesperson or customer) and the context of their statements."
    )

    agent_objective = "Objective: Your critical assessments aim to refine the sales teams approach, enhancing their ability to engage effectively with clients and close sales successfully. Your feedback will serve as a valuable tool for ongoing training and development within the sales department.KEEP FEEDBACK SHORT"

    chat_completion = client.chat.completions.create(
    messages=[
          {"role": "system", "content": agent_role},
          {"role": "user", "content": agent_objective},
          {"role": "assistant", "content": agent_considerations},
          {"role": "user", "content": agent_responsibility},
          {"role": "assistant", "content": "Sure, please provide the transcript."},
          {"role": "user", "content": text1}
    ],
    model="mixtral-8x7b-32768",
    seed=1212,
    temperature=0.0001,
    )
    return chat_completion.choices[0].message.content

async def speech_to_text(audio_file_path, processing_model):

    # Function 1: Convert speech to text
    if processing_model == 'OpenAI':
      transcribed_text = transcribe_with_openai_whisper(audio_file_path)
    else:
      transcribed_text = transcribe_with_local(audio_file_path)

    # Function 2: Translate to English
    translated_text = translate_to_english(transcribed_text)

    # Function 3: Summarize using openAI
    summary = summarize_text_groq(translated_text)

    # Return the processed text
    return summary

def speech_text(audio_file_path, processing_option):
    return asyncio.run(speech_to_text(audio_file_path, processing_option))

# Set the title and header
st.title("Sale - Call to summary Demo App")
st.header("Upload an Vietnamese audio file to process:")

# Select the transcription model
processing_option = st.radio(
    "Choose a transcription option:",
    ('Local', 'OpenAI')
)
# Upload audio file
uploaded_file = st.file_uploader("Choose Vietnamese audio file")


# Process uploaded file
# Process uploaded file
if uploaded_file is not None:
    # Read the content of the uploaded file
    file_content = uploaded_file.read()

    # Use magic to detect the file type
    file_type = magic.from_buffer(file_content, mime=True)

    # Assign the correct extension based on the detected file type
    if file_type == "audio/mpeg":
        extension = ".mp3"
    elif file_type == "audio/x-wav":
        extension = ".wav"
    else:
        raise ValueError("Unsupported file type")

    # Save the audio file with the correct extension
    filename = str(uuid.uuid4()) + extension
    with open(filename, "wb") as f:
        f.write(file_content)

    # Process the audio file
    processed_text = speech_text(filename, processing_option)

    # Display the processed text
    st.write("Processed Text:")
    st.write(processed_text)
####################################################

Overwriting app.py
